# Pay to win: generate loads of art in record time using the openai image generation API

## load secret and instlal libs

In [6]:
# --- 1) Load key from colab secrets (must be set in "Secrets" panel) ---
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# --- 2) Install & import client ---
!pip install --quiet openai python-slugify

## define image generation and saving function

In [12]:
from openai import OpenAI
import base64
from datetime import datetime
from slugify import slugify
import os

SAVE_DIR = "/content/drive/MyDrive/AI/dalle3"
os.makedirs(SAVE_DIR, exist_ok=True)   # ensure path exists

client = OpenAI()

def save_image_from_prompt(prompt: str) -> str:
    """
    Generate an image from a prompt using gpt-image-1,
    save it as PNG with timestamp + slug (first+last 50 chars)
    under /content/drive/MyDrive/AI/dalle3, and return the filename.
    """
    # --- Call API ---
    img = client.images.generate(
        model="gpt-image-1",
        prompt=prompt,
        n=1,
        size="1024x1024"
    )

    # --- Build filename ---
    def slug_first_last(text: str, n: int = 50) -> str:
        text = (text or "").strip()
        if len(text) <= 2*n:
            snippet = text
        else:
            snippet = text[:n] + " " + text[-n:]
        return slugify(snippet) or "image"

    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    slug = slug_first_last(prompt, 50)
    fname = f"{timestamp}-{slug}.png"
    fpath = os.path.join(SAVE_DIR, fname)

    # --- Write file ---
    image_bytes = base64.b64decode(img.data[0].b64_json)
    with open(fpath, "wb") as f:
        f.write(image_bytes)

    print(f"Saved → {fpath}")
    return fpath


Example run (deactivated to go faster)

In [11]:
# save_image_from_prompt('a delicious pizza with large zucchini flowers and blue cheese')

Saved → 20251019-001203-a-delicious-pizza-with-large-zucchini-flowers-and-blue-cheese.png


'20251019-001203-a-delicious-pizza-with-large-zucchini-flowers-and-blue-cheese.png'

## List of prompts

In [18]:
import glob
import os

base_dir = "/content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists"

# **Recursive glob: use "**" + recursive=True**
csv_files = glob.glob(os.path.join(base_dir, "**", "*.csv"), recursive=True)
txt_files = glob.glob(os.path.join(base_dir, "**", "*.txt"), recursive=True)

print("CSV files:")
for f in csv_files:
    print(" -", f)

print("\nTXT files:")
for f in txt_files:
    print(" -", f)


CSV files:
 - /content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists/todo2/b6/good_taste_prompts.csv
 - /content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists/todo2/b6/precious_still_lifes_prompt.csv
 - /content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists/todo2/b7/mosaic_prompts.csv
 - /content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists/todo2/b7/poignant_future_prompts.csv
 - /content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists/todo2/b7/sartphone_proof_of_concept_prompts.csv
 - /content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists/todo/prompts2.csv
 - /content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists/todo/prompts3.csv
 - /content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists/todo/prompt4.csv
 - /content/drive/MyDrive/AI/deliberate_model/previous

In [28]:
csv_files[20]

'/content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists/done/b5/thomas_cole_posthumous_prompts.csv'

In [22]:
pd.read_csv(csv_files[0])

,artist,prompt
0,Rembrandt van Rijn,"impressive oil painting, intimate portrait of ..."
1,Michelangelo,"awe-inspiring fresco, depiction of the creatio..."
2,Artemisia Gentileschi,"powerful oil painting, portrayal of a strong f..."
3,Leonardo da Vinci,"intriguing oil painting, enigmatic portrait of..."
4,Henri Matisse,"vibrant oil painting, lively interior scene wi..."
5,Katsushika Hokusai,"striking woodblock print, dramatic seascape wi..."
6,Georgia O'Keeffe,"sensual oil painting, close-up of a blooming f..."
7,Vincent van Gogh,"expressive oil painting, starry night sky over..."
8,Jackson Pollock,"energetic action painting, large-scale abstrac..."
9,Yayoi Kusama,"immersive installation, mirrored room filled w..."


In [29]:
import glob
import os
import json
import pandas as pd

base_dir = "/content/drive/MyDrive/AI/deliberate_model/previous images/gpt batches/prompt lists"

files = glob.glob(os.path.join(base_dir, "**", "*.csv"), recursive=True)
files += glob.glob(os.path.join(base_dir, "**", "*.txt"), recursive=True)

rows = []

def try_parse_json(text):
    try:
        return json.loads(text)
    except:
        return None

for f in files:
    ext = os.path.splitext(f)[1].lower()

    # CSV: only keep prompt column (ignore everything else)
    if ext == ".csv":
        try:
            df_csv = pd.read_csv(f)
            if "prompt" in df_csv.columns:
                for p in df_csv["prompt"].dropna().tolist():
                    rows.append({"filename": f, "prompt": str(p)})
                continue
            # else: no prompt col — skip this CSV entirely
            continue
        except:
            pass  # fallback to txt logic if badly formed CSV

    # TXT / fallback: parse JSON list-of-dicts or treat as whole text
    with open(f, "r", encoding="utf-8", errors="ignore") as fh:
        text = fh.read()

    data = try_parse_json(text)
    if isinstance(data, list) and all(isinstance(x, dict) for x in data):
        for item in data:
            p = item.get("prompt")
            if p:
                rows.append({"filename": f, "prompt": str(p)})
        continue

    # not JSON — treat entire text as 1 prompt entry
    rows.append({"filename": f, "prompt": text})

df = pd.DataFrame(rows, columns=["filename", "prompt"])
df


,filename,prompt
0,/content/drive/MyDrive/AI/deliberate_model/pre...,"impressive oil painting, intimate portrait of ..."
1,/content/drive/MyDrive/AI/deliberate_model/pre...,"awe-inspiring fresco, depiction of the creatio..."
2,/content/drive/MyDrive/AI/deliberate_model/pre...,"powerful oil painting, portrayal of a strong f..."
3,/content/drive/MyDrive/AI/deliberate_model/pre...,"intriguing oil painting, enigmatic portrait of..."
4,/content/drive/MyDrive/AI/deliberate_model/pre...,"vibrant oil painting, lively interior scene wi..."
...,...,...
227,/content/drive/MyDrive/AI/deliberate_model/pre...,"""prompt""\n""Digital painting, a bustling cyberp..."
228,/content/drive/MyDrive/AI/deliberate_model/pre...,"""prompt""\n""Digital painting, a dystopian citys..."
229,/content/drive/MyDrive/AI/deliberate_model/pre...,"""prompt""\n""Oil painting, a dense European fore..."
230,/content/drive/MyDrive/AI/deliberate_model/pre...,"""prompt""\n""Oil painting, a quaint English cott..."


In [30]:
import os
import pandas as pd
from tqdm.auto import tqdm

out_dir = "/content/drive/MyDrive/AI/dalle3"
os.makedirs(out_dir, exist_ok=True)

log_path = os.path.join(out_dir, "generated_prompts.csv")

# --- Initialize CSV file with header once if not exists
if not os.path.exists(log_path):
    pd.DataFrame(columns=["prompt", "file", "error"]).to_csv(log_path, index=False)

for row in tqdm(df.itertuples(), total=len(df), desc="Generating"):
    prompt = row.prompt
    try:
        fname = save_image_from_prompt(prompt)
        err = None
    except Exception as e:
        fname = None
        err = str(e)

    # --- Append immediately (1 row)
    pd.DataFrame([{
        "prompt": prompt,
        "file": fname,
        "error": err
    }]).to_csv(log_path, mode="a", index=False, header=False)


Generating:   0%|          | 0/232 [00:00<?, ?it/s]

Saved → /content/drive/MyDrive/AI/dalle3/20251019-003857-impressive-oil-painting-intimate-portrait-of-an-a-dutch-golden-age-emotionally-resonant-masterpiece.png
Saved → /content/drive/MyDrive/AI/dalle3/20251019-004024-powerful-oil-painting-portrayal-of-a-strong-femal-ntileschi-meets-baroque-art-inspiring-canvas-work.png
Saved → /content/drive/MyDrive/AI/dalle3/20251019-004112-intriguing-oil-painting-enigmatic-portrait-of-a-n-s-the-italian-renaissance-captivating-masterpiece.png
Saved → /content/drive/MyDrive/AI/dalle3/20251019-004138-vibrant-oil-painting-lively-interior-scene-with-b-enri-matisse-meets-fauvism-enchanting-canvas-work.png
Saved → /content/drive/MyDrive/AI/dalle3/20251019-004236-striking-woodblock-print-dramatic-seascape-with-c-ushika-hokusai-meets-ukiyo-e-breathtaking-artwork.png
Saved → /content/drive/MyDrive/AI/dalle3/20251019-004326-sensual-oil-painting-close-up-of-a-blooming-flowe-ffe-meets-american-modernism-alluring-canvas-work.png
Saved → /content/drive/MyDrive/AI/